<a href="https://colab.research.google.com/github/ThomasMoming/IAT460-A2/blob/main/460_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 安装 tracery（用于 CFG 生成）
!pip install tracery

from collections import defaultdict
import random
import tracery
from tracery.modifiers import base_english


  Preparing metadata (setup.py) ... done
  Created wheel for tracery: filename=tracery-0.1.1-py3-none-any.whl size=7681 sha256=17b3bcd711ef279d02a3d7bf7b02263a70f08a4bac4520a14452c394533653a6
  Stored in directory: /root/.cache/pip/wheels/01/7e/b2/55c94c8ee93425fdc7a582c824ae57abd8e0b0d55a55c4ee9b
Successfully built tracery


In [31]:
import numpy as np


In [40]:
#构建CFG
rules = {
    "start": [
        "#act1# #act2# #act3#"
    ],

    # 第一幕：主角回归
    "act1": [
        "#protagonist# has been in hiding for years, but today, he finally returns to Vancouver. #discovery#. He clenches his fist, eyes cold as ice: '#first_reaction#'"
    ],
    "protagonist": ["Wilson, the Shadow Emperor of Canada"],
    "discovery": [
        "His family was humiliated by the local mafia",
        "His fiancée Via was forced into marriage by a consortium",
        "His old rival has taken control of the entire business empire",
        "His old friend betrayed him and took away his inheritance",
        "His family's wealth was stolen and they were left in ruins"
    ],
    "first_reaction": [
        "'Those who dare harm my family will pay the price!'",
        "'Dare to harm my people? You are seeking death!'",
        "'From today, the world will remember my name!'",
        "'I swore never to look back, but you forced me…'",
    ],

    # 第二幕：觉醒、反击
    "act2": [
        "Years of patience and restraint finally came to an end. {BUSINESS}, shocking everyone. The enemy sneers: '#villain_mock#'.Just as everyone thought he would fail, Mr. Little Potato appeared beside him and called out 'Boss!"
    ],
    "villain_mock": [
        "'A washed-up fool like you dares to return? Hilarious!'",
        "'Once a loser, always a loser. Get lost!'",
        "'You think you can challenge me? Know your place!'",
        "'Pathetic. You’re nothing but a joke to us.'",
        "'Did you come back just to embarrass yourself again?'"
    ],

    # 第三幕：最终胜利
    "act3": [
        "In the end, he {REVENGE}, leaving the world in shock. #result#"
    ],
    "result": [
        "The audience fell to their knees, unable to raise their heads.",
        "His enemies now trembled and begged for mercy with tears in their eyes.",
        "He turned and walked away, leaving behind his legend - untouchable, unbeatable, unrivaled."
    ]
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

# 生成基本剧情框架
story_skeleton = grammar.flatten("#start#")
print("基础剧情结构:", story_skeleton)

基础剧情结构: Wilson, the Shadow Emperor of Canada has been in hiding for years, but today, he finally returns to Vancouver. His family was humiliated by the local mafia. He clenches his fist, eyes cold as ice: ''Those who dare harm my family will pay the price!'' Years of patience and restraint finally came to an end. {BUSINESS}, shocking everyone. The enemy sneers: ''A washed-up fool like you dares to return? Hilarious!''.Just as everyone thought he would fail, Mr. Little Potato appeared beside him and called out 'Boss! In the end, he {REVENGE}, leaving the world in shock. His enemies now trembled and begged for mercy with tears in their eyes.


In [35]:
def build_markov_chain(text, order=3):
    chain = defaultdict(list)
    words = text.split()

    for i in range(len(words) - order):
        state = tuple(words[i:i+order])
        next_word = words[i+order]
        chain[state].append(next_word)

    return chain

def generate_weighted_text(chain, num_words=50):
    words = list(random.choice(list(chain.keys())))
    order = len(list(chain.keys())[0])

    for _ in range(num_words - order):
        state = tuple(words[-order:])
        if state in chain:
            next_words = chain[state]
            word_counts = {word: next_words.count(word) for word in set(next_words)}
            words_list, weights = zip(*word_counts.items())
            probabilities = np.array(weights) / sum(weights)  # 归一化概率
            next_word = np.random.choice(words_list, p=probabilities)
            words.append(next_word)
        else:
            break

    return ' '.join(words)


# 训练数据（你可以替换成更长的龙王小说文本）
sample_text = """
After ten years of exile, Wilson stood at the entrance of the grand city that had once cast him aside. His family, once a dominant force, had been trampled underfoot by their enemies.

"Look who dares to return," a sneering voice called. It was Logan Carter, the man who had orchestrated his family's downfall. "You should have stayed in the shadows."

Wilson’s gaze was cold, his aura overwhelming. "Ten years ago, I swore I'd reclaim everything. Those who wronged me will kneel before me."

Carter laughed. "You? A useless exile? What can you do?"

A loud crack echoed as Wilson stepped forward. The air itself seemed to ripple with his presence. "Kneel," he commanded.

To everyone's shock, Carter trembled and collapsed to his knees. The once-mighty tyrant was now nothing more than a shivering coward.


Wilson entered the grand wedding hall, where his fiancée, the woman who had sworn to wait for him, was about to marry another man.

"How dare you show your face here?" Lucas Grey, the wealthy heir, smirked from the altar. "She's mine now."

Wilson’s eyes burned with fury. He stepped forward, his mere presence sending shivers through the crowd. "You think you can take what belongs to me?"

Before Lucas could react, Wilson flicked his wrist. A gust of unseen force knocked Lucas to the ground, his expensive suit now covered in dust.

"From this moment on," Wilson declared, "anyone who dares touch what’s mine will regret it."

The corporate elites laughed as Wilson walked into the conference room. "So, the beggar has returned?" mocked Edward Langley, the CEO of Langley Enterprises.


Wilson smirked, his fingers tapping on the document in front of him. "This is the contract that dissolves your company, effective immediately."

Langley's smile froze. "That's impossible!"

"Is it?" Wilson raised an eyebrow. "When I left, I didn’t just disappear. I built an empire. And now, I’m taking back what’s rightfully mine."

Langley collapsed into his chair, his face pale. Wilson turned, walking away as the room filled with whispers of his terrifying comeback.


At the grand auction, the priceless via was about to be sold to a wealthy magnate when Wilson stepped in.

"That artifact belongs to my family," he said coldly.

The magnate sneered. "Everything has a price, boy."

Wilson chuckled and tossed a black card onto the table. "Then I'll buy it. And while I'm at it, I'll buy this entire auction house."

The room fell silent. Even the wealthiest elites couldn't afford such extravagance. The magnate's face twisted in disbelief as the auctioneer stammered, "S-Sold!"


Wilson walked into the underground fighting ring, where champions had reigned undefeated for years.

"You think you can challenge me?" growled Brutus, the reigning champion.

Wilson cracked his knuckles. "I don’t think. I know."

The fight was over in seconds. Wilson didn't just defeat Brutus—he humiliated him. The once-mighty warrior lay sprawled on the ground, gasping for breath.

From that day on, no one dared to question Wilson’s strength.


Wilson's younger sister had been bullied relentlessly by the wealthy and powerful. Today, he would end it.

Walking into the elite private school, he found the ringleader, Daniel Mercer.

"You’ve been hurting my family," Wilson said, his voice like steel.

Mercer scoffed. "So what? What can you do about it?"

Wilson smiled coldly. "Apologize."

"Or what?"

A loud crack echoed as Wilson grabbed Mercer’s wrist and twisted. The arrogant young master screamed, begging for mercy.

"From now on," Wilson said, "no one touches my family."



"Wilson? Never heard of him."

That was the response of the elite socialites when Wilson returned to the city.

By the end of the week, his name was on everyone's lips.

Within days, he had taken over the largest corporation, crushed his old enemies, and reclaimed his family estate.

"Still never heard of me?" he asked his former foes, who now knelt before him.


A waiter in a five-star restaurant spilled wine on Wilson, laughing as he mocked, "You can't afford this place anyway."

Wilson smiled. "Is that so?"

Minutes later, the restaurant's owner rushed in, bowing. "Sir! I had no idea you were dining here. This establishment is now yours."

The waiter's face turned ghostly pale as he stammered, "I-I was just joking!"

Wilson took his time sipping his wine. "So was I."


Accused of a crime he didn’t commit, Wilson was dragged to court. The prosecution smirked, convinced they had won.

Then Wilson handed the judge a document. "This proves my innocence. And more than that, it proves the real culprit."

The entire courtroom gasped as the real criminal, sitting in the audience, turned pale.

"Case dismissed," the judge announced.

Wilson walked away, his enemies now trapped in their own schemes.



A lavish party was being held by the city’s elites when Wilson stepped through the doors.

"Who let the beggar in?" someone sneered.

Wilson smirked. "Beggar? Look again."

He tossed a document onto the table. It was a deed—he now owned their entire estate.

The sneers turned to gasps as everyone realized they had mocked the wrong man.


"""

# 构建马尔可夫链
chain = build_markov_chain(sample_text, order=3)




In [42]:
battle_text = generate_weighted_text(chain, num_words=50)
business_text = generate_weighted_text(chain, num_words=50)
revenge_text = generate_weighted_text(chain, num_words=50)
story_skeleton = grammar.flatten("#start#")

# 使用占位符填充 Markov Chain 生成的细节
final_story = (story_skeleton
               .replace("{BATTLE}", battle_text)
               .replace("{BUSINESS}", business_text)
               .replace("{REVENGE}", revenge_text))

print("\nFinal Story:\n", final_story)



Final Story:
 Wilson, the Shadow Emperor of Canada has been in hiding for years, but today, he finally returns to Vancouver. His old friend betrayed him and took away his inheritance. He clenches his fist, eyes cold as ice: ''I swore never to look back, but you forced me…'' Years of patience and restraint finally came to an end. Langley Enterprises. Wilson smirked, his fingers tapping on the document in front of him. "This is the contract that dissolves your company, effective immediately." Langley's smile froze. "That's impossible!" "Is it?" Wilson raised an eyebrow. "When I left, I didn’t just disappear. I built an empire. And now, I’m taking, shocking everyone. The enemy sneers: ''You think you can challenge me? Know your place!''.Just as everyone thought he would fail, Mr. Little Potato appeared beside him and called out 'Boss! In the end, he stepped through the doors. "Who let the beggar in?" someone sneered. Wilson smirked. "Beggar? Look again." He tossed a document onto the tab

In [43]:
# 生成多个故事
num_stories = 5  # 生成 5 个不同的故事

for i in range(num_stories):
    # 重新生成故事骨架
    story_skeleton = grammar.flatten("#start#")

    # 重新生成 Markov Chain 生成的细节文本
    battle_text = generate_weighted_text(chain, num_words=50)
    business_text = generate_weighted_text(chain, num_words=50)
    revenge_text = generate_weighted_text(chain, num_words=50)

    # 使用占位符填充 Markov Chain 生成的细节
    final_story = (story_skeleton
                   .replace("{BATTLE}", battle_text)
                   .replace("{BUSINESS}", business_text)
                   .replace("{REVENGE}", revenge_text))

    # 打印每个故事
    print(f"\nFinal Story {i+1}:\n")
    print(final_story)
    print("\n" + "="*80 + "\n")  # 分隔不同的故事



Final Story 1:

Wilson, the Shadow Emperor of Canada has been in hiding for years, but today, he finally returns to Vancouver. His fiancée Via was forced into marriage by a consortium. He clenches his fist, eyes cold as ice: ''From today, the world will remember my name!'' Years of patience and restraint finally came to an end. Wilson walked into the conference room. "So, the beggar has returned?" mocked Edward Langley, the CEO of Langley Enterprises. Wilson smirked, his fingers tapping on the document in front of him. "This is the contract that dissolves your company, effective immediately." Langley's smile froze. "That's impossible!" "Is it?" Wilson raised, shocking everyone. The enemy sneers: ''Did you come back just to embarrass yourself again?''.Just as everyone thought he would fail, Mr. Little Potato appeared beside him and called out 'Boss! In the end, he of the elite socialites when Wilson returned to the city. By the end of the week, his name was on everyone's lips. Within d